# Breast cancer diagnosis classification with scikit-learn (save model explanations via AML Run History)

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/explain-tabular-data-run-history/explain-run-history-sklearn-classification.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

Explain a model with the AML explain-model package

1. Train a SVM classification model using Scikit-learn
2. Run 'explain_model' with AML Run History, which leverages run history service to store and manage the explanation data

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn import svm
from azureml.explain.model.tabular_explainer import TabularExplainer

# 1. Run model explainer locally with full data

## Load the breast cancer diagnosis data

In [ ]:
breast_cancer_data = load_breast_cancer()
classes = breast_cancer_data.target_names.tolist()

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(breast_cancer_data.data, breast_cancer_data.target, test_size=0.2, random_state=0)

## Train a SVM classification model, which you want to explain

In [ ]:
clf = svm.SVC(gamma=0.001, C=100., probability=True)
model = clf.fit(x_train, y_train)

## Explain predictions on your local machine

In [ ]:
tabular_explainer = TabularExplainer(model, x_train, features=breast_cancer_data.feature_names, classes=classes)

## Explain overall model predictions (global explanation)

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = tabular_explainer.explain_global(x_test)

# 2. Save Model Explanation With AML Run History

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, Run
from azureml.explain.model.tabular_explainer import TabularExplainer
from azureml.contrib.explain.model.explanation.explanation_client import ExplanationClient
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
experiment_name = 'explain_model'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()
client = ExplanationClient.from_run(run)

In [ ]:
# Uploading model explanation data for storage or visualization in webUX
# The explanation can then be downloaded on any compute
client.upload_model_explanation(global_explanation)

In [ ]:
# Get model explanation data
explanation = client.download_model_explanation()
local_importance_values = explanation.local_importance_values
expected_values = explanation.expected_values

In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
explanation = client.download_model_explanation(top_k=4)
global_importance_values = explanation.get_ranked_global_values()
global_importance_names = explanation.get_ranked_global_names()
per_class_names = explanation.get_ranked_per_class_names()[0]
per_class_values = explanation.get_ranked_per_class_values()[0]

In [ ]:
print('per class feature importance values: {}'.format(per_class_values))
print('per class feature importance names: {}'.format(per_class_names))

In [ ]:
dict(zip(per_class_names, per_class_values))